In [5]:
#label:korea로 크롤링
import csv
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time

base_url = 'https://songdata.io/'
search_url = urljoin(base_url, 'search?query=label%3Akorea')

# 데이터를 저장할 CSV 파일을 엽니다.
with open('output.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    # 헤더 행을 작성합니다.
    writer.writerow(['Name', 'Artist', 'Key', 'BPM'])
    
    url = search_url
    while url:
        response = requests.get(url)

        if response.status_code == 200:
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')

            # 테이블의 각 행에 대한 정보를 가져옵니다.
            rows = soup.select('tr.table_object')
            
            for row in rows:
                # 각 행에서 필요한 정보를 추출합니다.
                name = row.select_one('td.table_name').text.strip()
                artist = row.select_one('td.table_artist').text.strip()
                key = row.select_one('td.table_key').text.strip()
                bpm = row.select_one('td.table_bpm').text.strip()

                # 추출한 정보를 CSV 파일에 작성합니다.
                writer.writerow([name, artist, key, bpm])
            
            # Next Page 버튼의 링크를 찾습니다.
            next_page = soup.select_one('span.search_page a')
            if next_page:
                url = urljoin(base_url, next_page['href'])
                # next 버튼을 누르기 전에 5초 동안 대기합니다.
                time.sleep(5)
            else:
                url = None


In [12]:
#멜론 크롤링 값으로 크롤링
import csv
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time

# CSV 파일을 열어 모든 행의 제목(title) 값을 가져옴
titles = []
with open('melon_crawling_id.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # 헤더 행을 건너뜀
    for row in reader:
        title = row[0]  # 제목(title) 값을 가져옴
        titles.append(title)

# 검색 URL을 생성하고 데이터를 저장할 CSV 파일을 염
base_url = 'https://songdata.io/'
search_url = urljoin(base_url, 'search?query=')

with open('bpm_bymelon.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    # 헤더 행을 작성합니다.
    writer.writerow(['Name', 'Artist', 'Key', 'BPM'])

    for title in titles:
        url = search_url + title  # 검색 URL을 생성

        try:
            response = requests.get(url)

            if response.status_code == 200:
                html = response.text
                soup = BeautifulSoup(html, 'html.parser')

                # 검색 결과가 있는지 확인.
                result = soup.select_one('tr.table_object')
                if result:
                    # 필요한 정보를 추출
                    name = result.select_one('td.table_name a').text.strip()
                    artist = result.select_one('td.table_artist').text.strip()
                    key = result.select_one('td.table_key').text.strip()
                    bpm = result.select_one('td.table_bpm').text.strip()

                    # 추출한 정보를 CSV 파일에 작성
                    writer.writerow([name, artist, key, bpm])
                else:
                    writer.writerow([title, 'Not Found', '', ''])
            else:
                writer.writerow([title, 'Error', '', ''])
        except requests.exceptions.RequestException as e:
            writer.writerow([title, 'Error', '', ''])
            print(f"오류 발생: {e}")


KeyboardInterrupt: 

In [9]:
# 기존 CSV 파일을 읽고 not found error인거 수정(bpm_bymelon.csv로 저장해둠)
with open('bpm_bymelon.csv', 'r', newline='', encoding='utf-8') as f:
    rows = list(csv.reader(f))
    for i in range(1, len(rows)):
        if rows[i][1] == 'Not Found' or rows[i][1] == 'Error':
            title = rows[i][0]
            url = search_url + title

            try:
                response = requests.get(url)

                if response.status_code == 200:
                    html = response.text
                    soup = BeautifulSoup(html, 'html.parser')
                    result = soup.select_one('tr.table_object')

                    if result:
                        name = result.select_one('td.table_name a').text.strip()
                        artist = result.select_one('td.table_artist').text.strip()
                        key = result.select_one('td.table_key').text.strip()
                        bpm = result.select_one('td.table_bpm').text.strip()

                        # 정보를 수정
                        rows[i] = [name, artist, key, bpm]
            except requests.exceptions.RequestException as e:
                print(f"오류 발생: {e}")

# 수정된 정보로 업데이트된 CSV 파일을 저장
with open('bpm_bymelon.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(rows)

In [41]:
import pandas as pd
import csv
with open('bpm_bymelon_수정덮어쓴거.csv', 'r', newline='', encoding='utf-8') as f:
    rows = list(csv.reader(f))
    header = rows[0]  # 헤더를 따로 저장
    data_rows = rows[1:]  # 데이터 행만 저장
    null_count = sum(1 for row in data_rows if 'Not Found' in row[1] or 'Error' in row[1])

print('덮어쓴 거 총 수:', len(data_rows))
print(f"'Not Found' 또는 'Error'가 포함된 행의 개수: {null_count}")
print()

data_samename = pd.read_csv('bpm_bymelon_수정덮어쓴거.csv', encoding='utf-8')
print('덮어쓴 거 총 수:',len(data_samename))
print('덮어쓴 거 결측값 여부 :',data_samename.isnull().sum())

# 'Artist' 열이 'Not Found'나 'Error'가 아니면서 'BPM'과 'Key' 열의 값이 모두 결측값인 행
rows_with_valid_artist_and_null_bpm_and_key = data_samename[
    (data_samename['Artist'] != 'Not Found') &
    (data_samename['Artist'] != 'Error') &
    data_samename[['BPM', 'Key']].isnull().all(axis=1)
]

# 'Artist' 열이 'Not Found'나 'Error'가 아니면서 'BPM'과 'Key' 열의 값이 모두 결측값인 행의 개수
count_valid_artist_and_null_bpm_and_key = len(rows_with_valid_artist_and_null_bpm_and_key)
print("'Artist'가 Not Found나 Error가 아니면서 'BPM'과 'Key' 열이 모두 결측값인 행의 개수:", count_valid_artist_and_null_bpm_and_key)

# 백색소음 이런게 bpm이 없음
# print("'Artist'가 Not Found나 Error가 아니면서 'BPM'과 'Key' 열이 모두 결측값인 행:")
# print(rows_with_valid_artist_and_null_bpm_and_key)

덮어쓴 거 총 수: 9000
'Not Found' 또는 'Error'가 포함된 행의 개수: 396

덮어쓴 거 총 수: 9000
덮어쓴 거 결측값 여부 : Name        0
Artist      0
Key       417
BPM       417
dtype: int64
'Artist'가 Not Found나 Error가 아니면서 'BPM'과 'Key' 열이 모두 결측값인 행의 개수: 21
